In [ ]:
from pyscf import gto, scf, mp, cc

a = 2 # bond length in a cluster
d = 10 # distance between each cluster
unit = 'b' # unit of length
na = 2  # size of a cluster (monomer)
nc = 5 # set as integer multiple of monomers
spin = 0 # spin per monomer
frozen = 0 # frozen orbital per monomer
elmt = 'H'
basis = 'sto6g'
atoms = ""
for n in range(nc*na):
    shift = ((n - n % na) // na) * (d-a)
    atoms += f"{elmt} {n*a+shift:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis=basis, spin=0, unit=unit, verbose=4, max_memory=16000)
mf = scf.UHF(mol).density_fit()
mf.kernel()

nfrozen = 0
# canonical
# mmp = mp.UMP2(mf, frozen=nfrozen)
# mmp.kernel()

# mycc = cc.UCCSD(mf, frozen=nfrozen)
# eris = mycc.ao2mo()
# mycc.kernel(eris=eris)
# et = mycc.ccsd_t(eris=eris)

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#5074-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Sat Dec  6 19:07:40 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 10
[INPUT] num. electrons = 10
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = b
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.0000000

In [2]:
from pyscf import lo
import numpy as np
orbocca = mf.mo_coeff[0][:,nfrozen:np.count_nonzero(mf.mo_occ[0])]
orbloca = lo.PipekMezey(mol, orbocca).kernel()
orboccb = mf.mo_coeff[1][:,nfrozen:np.count_nonzero(mf.mo_occ[1])]
orblocb = lo.PipekMezey(mol, orboccb).kernel()
lo_coeff = [orbloca, orblocb]

oa = [[[i],[]] for i in range(orbloca.shape[1])]
ob = [[[],[i]] for i in range(orblocb.shape[1])]
frag_lolist = oa + ob



******** <class 'pyscf.lo.pipek.PipekMezey'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.05
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3
init_guess = atomic
pop_method = meta_lowdin
Set conv_tol_grad to 0.000316228
macro= 1  f(x)= 2.4999999627851  delta_f= 2.5  |g|= 0.000624393  1 KF 1 Hx
macro= 2  f(x)= 2.4999999627851  delta_f= -8.88178e-16  |g|= 2.12414e-11  1 KF 1 Hx
macro X = 2  f(x)= 2.4999999627851  |g|= 2.12414e-11  4 intor 2 KF 2 Hx


******** <class 'pyscf.lo.pipek.PipekMezey'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.05
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3
init_guess = atomic
pop_method = meta_lowdin


In [ ]:
import os, pickle
from ad_afqmc import config
def run_afqmc(options,nproc=None,
              option_file='options.bin'):

    with open(option_file, 'wb') as f:
        pickle.dump(options, f)

    use_gpu = options["use_gpu"]
    if use_gpu:
        print(f'# running AFQMC on GPU')
        config.afqmc_config = {"use_gpu": True}
        config.setup_jax()
        gpu_flag = "--use_gpu"
        mpi_prefix = ""
    else:
        print(f'# running AFQMC on CPU')
        gpu_flag = ""
        mpi_prefix = "mpirun "
        if nproc is not None:
            mpi_prefix += f"-np {nproc} "

    script='/home/yichi/research/software/cs_afqmc/ad_afqmc/lno_afqmc/lno_afqmc_ccsd_pt/run_lnoafqmc.py'

    # path = os.path.abspath(__file__)
    # dir_path = os.path.dirname(path)
    # script = f"{dir_path}/{script}"
    print(f'# AFQMC script: {script}')
    
    os.system(
        f"export OMP_NUM_THREADS=1; export MKL_NUM_THREADS=1;"
        f"{mpi_prefix} python {script} {gpu_flag} |tee afqmc.out"
    )

In [ ]:
from collections.abc import Iterable
from pyscf.lno import ulnoccsd
from ad_afqmc.lno_afqmc import ulno_afqmc

def run_lnoafqmc(mf,
                 options,
                 lo_coeff,
                 frag_lolist,
                 frozen = 0,
                 thresh = 1e-6,
                 chol_cut = 1e-5,
                 lno_type = ['1h']*2,
                 run_frg_list = None,
                 nproc = None):
    mlno = ulnoccsd.ULNOCCSD_T(mf, lo_coeff, frag_lolist, frozen=frozen).set(verbose=4)
    thresh = 1e-4
    mlno.lno_thresh = [thresh*10,thresh]
    lno_thresh = mlno.lno_thresh
    lno_type = ['1h','1h'] if lno_type is None else lno_type
    lno_thresh = [1e-5, 1e-6] if lno_thresh is None else lno_thresh
    lno_pct_occ = None
    lno_norb = None
    lo_proj_thresh = 1e-10
    lo_proj_thresh_active = 0.1
    eris = None

    if run_frg_list is not None:
        frag_lolist = [frag_lolist[i] for i in run_frg_list]

    nfrag = len(frag_lolist)
    if lno_pct_occ is None:
        lno_pct_occ = [None, None]
    if lno_norb is None:
        lno_norb = [[None,None]] * nfrag
    mf = mlno._scf

    if eris is None: eris = mlno.ao2mo()

    # Loop over fragment
    # frag_res = [None] * nfrag
    for ifrag, loidx in enumerate(frag_lolist):
        if len(loidx) == 2 and isinstance(loidx[0], Iterable): # Unrestricted
            orbloc = [lo_coeff[0][:,loidx[0]], lo_coeff[1][:,loidx[1]]]
            lno_param = [
                [
                    {
                        'thresh': (
                            lno_thresh[i][s] if isinstance(lno_thresh[i], Iterable)
                            else lno_thresh[i]
                        ),
                        'pct_occ': (
                            lno_pct_occ[i][s] if isinstance(lno_pct_occ[i], Iterable)
                            else lno_pct_occ[i]
                        ),
                        'norb': (
                            lno_norb[ifrag][i][s] if isinstance(lno_norb[ifrag][i], Iterable)
                            else lno_norb[ifrag][i]
                        ),
                    } for i in [0, 1]
                ] for s in range(2)
            ]

        else:
            orbloc = lo_coeff[:,loidx]
            lno_param = [{'thresh': lno_thresh[i], 'pct_occ': lno_pct_occ[i],
                            'norb': lno_norb[ifrag][i]} for i in [0,1]]

        lno_coeff, frozen, uocc_loc, frag_msg = mlno.make_las(eris, orbloc, lno_type, lno_param)
        # frag_res[ifrag], frag_msg = mlno.impurity_solve(mf, mo_coeff, uocc_loc, eris, frozen=frozen)

        mo_occ = mlno.mo_occ
        frozen, maskact = ulnoccsd.get_maskact(frozen, [mo_occ[0].size, mo_occ[1].size])
        mcc = ulnoccsd.UCCSD(mf, mo_coeff=lno_coeff, frozen=frozen).set(verbose=mlno.verbose_imp)
        mcc._s1e = mlno._s1e
        mcc._h1e = mlno._h1e
        mcc._vhf = mlno._vhf
        if mlno.kwargs_imp is not None:
            mcc = mcc.set(**mlno.kwargs_imp)

        (emp2,eccsd,ept), t1, t2, prjlo =\
            ulno_afqmc.ulno_ccsd(mcc, lno_coeff, uocc_loc, mo_occ, maskact, ccsd_t=True)
        
        from mpi4py import MPI
        if not MPI.Is_finalized():
            MPI.Finalize()
        
        ulno_afqmc.prep_afqmc(
            mf,lno_coeff,t1,t2,frozen,prjlo,
            options,chol_cut=chol_cut)
        
        run_afqmc(options,nproc)
        os.system(f'mv afqmc.out lnoafqmc.out{ifrag+1}')


In [7]:
options = {'n_eql': 5,
        'n_prop_steps': 50,
        'n_ene_blocks': 1,
        'n_sr_blocks': 10,
        'n_blocks': 10,
        'n_walkers': 1,
        'seed': 98,
        'walker_type': 'uhf',
        'trial': 'uccsd_pt_ad',
        'dt':0.005,
        'free_projection':False,
        'ad_mode':None,
        'use_gpu': False,
        'max_error': 1e-4
        }
run_lnoafqmc(mf,options,lo_coeff,frag_lolist,run_frg_list = [0])

LO occ proj: 1 active | 0 standby | 4 orthogonal
LO occ proj: 0 active | 0 standby | 5 orthogonal

WARN: CCSD detected DF being used in the HF object. MO integrals are computed based on the DF 3-index tensors.
It's recommended to use dfccsd.CCSD for the DF-CCSD calculations

# Calculating Effective Active Space One-electron Integrals
# Generating Cholesky Integrals
# Composing AO ERIs from DF basis
# Finished calculating Cholesky integrals
# Size of the correlation space
# Number of electrons: (1, 1)
# Number of basis functions: (2, 2)
# Alpha Basis Cholesky shape: (90, 2, 2)
#  Beta Basis Cholesky shape: (90, 2, 2)


In [5]:
import os, pickle
from ad_afqmc import config
def run_afqmc(options,nproc=None,
              option_file='options.bin'):

    with open(option_file, 'wb') as f:
        pickle.dump(options, f)

    use_gpu = options["use_gpu"]
    if use_gpu:
        print(f'# running AFQMC on GPU')
        config.afqmc_config = {"use_gpu": True}
        config.setup_jax()
        gpu_flag = "--use_gpu"
        mpi_prefix = ""
    else:
        print(f'# running AFQMC on CPU')
        gpu_flag = ""
        mpi_prefix = "mpirun "
        if nproc is not None:
            mpi_prefix += f"-np {nproc} "

    script='/home/yichi/research/software/cs_afqmc/ad_afqmc/lno_afqmc/lno_afqmc_ccsd_pt/run_ulnoafqmc.py'

    # path = os.path.abspath(__file__)
    # dir_path = os.path.dirname(path)
    # script = f"{dir_path}/{script}"
    print(f'# AFQMC script: {script}')
    
    os.system(
        f"export OMP_NUM_THREADS=1; export MKL_NUM_THREADS=1;"
        f"{mpi_prefix} python {script} {gpu_flag} |tee lno_afqmc.out"
    )

In [8]:
run_afqmc(options,nproc=1)

# running AFQMC on CPU
# AFQMC script: /home/yichi/research/software/cs_afqmc/ad_afqmc/lno_afqmc/lno_afqmc_ccsd_pt/run_ulnoafqmc.py
# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: (2, 2)
# nelec: (1, 1)
#
# n_eql: 5
# n_prop_steps: 50
# n_ene_blocks: 1
# n_sr_blocks: 10
# n_blocks: 10
# n_walkers: 1
# seed: 98
# walker_type: uhf
# trial: uccsd_pt_ad
# dt: 0.005
# free_projection: False
# use_gpu: False
# max_error: 0.0001
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
#
# 

# Propagating with 1 walkers
# Initial energy -5.290769
# Equilibration sweeps:
#   Iter 	 <e0> 	 	 <e0>_orb 	 <e012>_orb 	 <t12>_orb 	 time
      0 	 0.000000 	 0.000000 	 -0.020030 	  0.000000 	 5.13
      1 	 -0.041719	-0.020859	-0.020860	0.020714 	 9.50 
      2 	 -0.017336

In [37]:
from ad_afqmc import config
config.setup_jax()
MPI = config.setup_comm()
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

ham_data, prop, trial, wave_data, sampler, options, _ = (
    ulno_afqmc._prep_afqmc())

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#


# norb: (2, 2)
# nelec: (1, 1)
#
# n_eql: 5
# n_prop_steps: 50
# n_ene_blocks: 1
# n_sr_blocks: 10
# n_blocks: 10
# n_walkers: 1
# seed: 98
# walker_type: uhf
# trial: uhf
# dt: 0.005
# free_projection: False
# use_gpu: False
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
#


In [38]:
import time
from jax import numpy as jnp
from jax import random
init_time = time.time()
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

### initialize propagation
seed = options["seed"]
init_walkers = None
# dm_up = jnp.array(wave_data["mo_coeff"][0] @ wave_data["mo_coeff"][0].T.conj())
# dm_dn = jnp.array(wave_data["mo_coeff"][1] @ wave_data["mo_coeff"][1].T.conj())
# trial_rdm1 = [dm_up, dm_dn]
trial_rdm1 = trial.get_rdm1(wave_data)
if "rdm1" not in wave_data:
    wave_data["rdm1"] = trial_rdm1
ham_data = trial._build_measurement_intermediates(ham_data, wave_data)
ham_data = prop._build_propagation_intermediates(ham_data, trial, wave_data)

prop_data = prop.init_prop_data(trial, wave_data, ham_data, init_walkers)
if jnp.abs(jnp.sum(prop_data["overlaps"])) < 1.0e-6:
    raise ValueError(
        "Initial overlaps are zero. Pass walkers with non-zero overlap."
    )
prop_data["key"] = random.PRNGKey(seed + rank)

prop_data["overlaps"] = trial.calc_overlap(prop_data["walkers"], wave_data)
prop_data["n_killed_walkers"] = 0
prop_data["pop_control_ene_shift"] = prop_data["e_estimate"]
print(prop_data["e_estimate"])
print(emf)

-5.290769166910058
-5.290769166910056


In [ ]:
import opt_einsum as oe
from jax import vmap
def _calc_ecorr(trial, walker_up, walker_dn, ham_data, wave_data):
    '''uhf correlation energy'''
    # <HF|H-E0|walker>/<HF|walker>
    nocca, noccb = trial.nelec
    # rot_h1a, rot_h1b = ham_data['rot_h1']
    rot_chola, rot_cholb = ham_data['rot_chol']
    GFa, GFb = trial._calc_green(walker_up, walker_dn, wave_data)
    lga = oe.contract('gij,jk->gik', rot_chola[:,:nocca,nocca:],
                    GFa.T[nocca:,:nocca], backend="jax")
    lgb = oe.contract('gij,jk->gik', rot_cholb[:,:noccb,noccb:],
                    GFb.T[noccb:,:noccb], backend="jax")
    tr_lga = oe.contract('gii->g',lga)
    tr_lgb = oe.contract('gii->g',lgb)
    lglg_aa = oe.contract('g,g->',tr_lga,tr_lga) - oe.contract('gij,gji->',lga,lga)
    lglg_ab = oe.contract('g,g->',tr_lga,tr_lgb)*2
    lglg_bb = oe.contract('g,g->',tr_lgb,tr_lgb) - oe.contract('gij,gji->',lgb,lgb)
    ecorr = 0.5*(lglg_aa + lglg_ab + lglg_bb)
    return jnp.real(ecorr)

def _calc_eorb(trial, walker_up, walker_dn, ham_data, wave_data):
    '''uhf correlation energy'''
    # <HF|H-E0|walker>/<HF|walker>
    nocca, noccb = trial.nelec
    # rot_h1a, rot_h1b = ham_data['rot_h1']
    prjloa, prjlob = wave_data['prjlo']
    rot_chola, rot_cholb = ham_data['rot_chol']
    GFa, GFb = trial._calc_green(walker_up, walker_dn, wave_data)
    lga = oe.contract('gij,jk->gik', rot_chola[:,:nocca,nocca:],
                    GFa.T[nocca:,:nocca], backend="jax")
    lgb = oe.contract('gij,jk->gik', rot_cholb[:,:noccb,noccb:],
                    GFb.T[noccb:,:noccb], backend="jax")
    tr_lga = oe.contract('gii->g',lga)
    tr_lgb = oe.contract('gii->g',lgb)
    lga_orb = oe.contract('gik,ik->g',lga,prjloa)
    lgb_orb = oe.contract('gik,ik->g',lgb,prjlob)
    lglg_aa = oe.contract('g,g->',lga_orb,tr_lga) - oe.contract('gij,gjk,ik->',lga,lga,prjloa)
    lglg_ab = oe.contract('g,g->',lga_orb,tr_lgb) + oe.contract('g,g->',tr_lga,lgb_orb)
    lglg_bb = oe.contract('g,g->',lgb_orb,tr_lgb) - oe.contract('gij,gjk,ik->',lgb,lgb,prjlob)
    eorb = 0.5*(lglg_aa + lglg_ab + lglg_bb)
    return jnp.real(eorb)

In [ ]:
import jax
from jax import lax, jvp, vmap
def _t_orb(trial, walker_up, walker_dn, wave_data: dict) -> complex:
    '''
    <HF|t1+t2|walker>_i 
    = (C_ia <HF|i+ a|walker>/<HF|walker> + C_iajb <HF|i+ j+ a b|walker>/<HF|walker>) * <HF|walker>
    = (C_ia G_ia + C_iajb (G_ia G_jb-G_ib G_ja)) * <HF|walker>
    prj onto spin-orbit i
    '''
    pa, pb = wave_data["prjlo"]
    nocca, noccb = trial.nelec
    t1a, t1b = wave_data["t1a"], wave_data["t1b"]
    t2aa, t2ab, t2bb = wave_data["t2aa"], wave_data["t2ab"], wave_data["t2bb"]
    greena, greenb = trial._calc_green(walker_up, walker_dn, wave_data)
    greena, greenb = greena[:nocca, nocca:], greenb[:noccb, noccb:]
    o0 = trial._calc_overlap(walker_up,walker_dn,wave_data)
    o1 = oe.contract("ia,ka,ik->", t1a, greena, pa, backend="jax") \
            + oe.contract("ia,ka,ik->", t1b, greenb, pa, backend="jax")
    o2 = (oe.contract("iajb,ka,jb,ik->", t2aa, greena, greena, pa, backend="jax")
            + oe.contract("iajb,ka,jb,ik->", t2bb, greenb, greenb, pb, backend="jax")
            + oe.contract("iajb,ka,jb,ik->", t2ab, greena, greenb, pa, backend="jax")
            + oe.contract("jbia,jb,ka,ik->", t2ab, greena, greenb, pb, backend="jax")) * 0.5
    return (o1 + o2) * o0

    
def _t_exp1_orb(trial, x, h1_mod, walker_up, walker_dn, wave_data):
        '''
        unrestricted t_ia <psi_i^a|exp(x*h1_mod)|walker>/<HF|walker>
        '''
        walker_up_1x = walker_up + x * h1_mod[0].dot(walker_up)
        walker_dn_1x = walker_dn + x * h1_mod[1].dot(walker_dn)
        olp = _t_orb(trial,walker_up_1x, walker_dn_1x, wave_data)
        return olp


def _t_exp2_orb(trial, x, chol_i, walker_up, walker_dn, wave_data) -> complex:
        '''
        t_ia <psi_i^a|exp(x*h2_mod)|walker>/<HF|walker>
        '''
        walker_up_2x = (
            walker_up
            + x * chol_i[0].dot(walker_up)
            + x**2 / 2.0 * chol_i[0].dot(chol_i[0].dot(walker_up))
        )
        walker_dn_2x = (
            walker_dn
            + x * chol_i[1].dot(walker_dn)
            + x**2 / 2.0 * chol_i[1].dot(chol_i[1].dot(walker_dn))
        )
        olp = _t_orb(trial,walker_up_2x,walker_dn_2x,wave_data)
        return olp

def _d2_exp2_orb_i(trial,chol_i,walker_up,walker_dn,wave_data):
        x = 0.0
        f = lambda a: _t_exp2_orb(trial,a,chol_i,walker_up,walker_dn,wave_data)
        _, d2f = jax.jvp(lambda x: jax.jvp(f, [x], [1.0])[1], [x], [1.0])
        return d2f


def _th12_orb(trial, walker_up, walker_dn, ham_data, wave_data):
        '''
        <HF|(t1+t2) (H-E0)|walker>/<HF|walker>
        '''
        norba, norbb = trial.norb
        chola, cholb = ham_data["chol"]
        chola = chola.reshape(-1, norba, norba)
        cholb = cholb.reshape(-1, norbb, norbb)
        chol = [chola, cholb]
        h1_mod = ham_data['h1_mod']
        h0_E0 = ham_data["h0"]-ham_data["E0"]

        # nocca, noccb = trial.nelec
        o0 = trial._calc_overlap(walker_up,walker_dn,wave_data)

        x = 0.0
        # one body
        f1 = lambda a: _t_exp1_orb(trial,a,h1_mod,walker_up,walker_dn,wave_data)
        olp_orb12, d_overlap = jvp(f1, [x], [1.0])

        # two body
        def scanned_fun(carry, c):
            walker_up, walker_dn, wave_data = carry
            return carry, _d2_exp2_orb_i(trial,c,walker_up,walker_dn,wave_data)

        _, d2_olp2_i = lax.scan(scanned_fun, (walker_up,walker_dn,wave_data), chol)
        d_2_overlap = jnp.sum(d2_olp2_i)/2

        # <hf|(t1+t2)_i (h0-E0+h1+h2)|walker>/<hf|walker>
        et_orb = (h0_E0*olp_orb12 + d_overlap + d_2_overlap) / o0
        t_orb = olp_orb12 /o0 # <(t1+t2)_i>

        return jnp.real(et_orb), jnp.real(t_orb)

In [48]:
walker_up, walker_dn = prop_data['walkers'][0][0], prop_data['walkers'][1][0]

In [42]:
t1a, t1b = t1
t2aa, t2ab, t2bb = t2
t2aa = (t2aa - t2aa.transpose(0, 1, 3, 2)) / 2
t2bb = (t2bb - t2bb.transpose(0, 1, 3, 2)) / 2
t2aa = t2aa.transpose(0, 2, 1, 3)
t2ab = t2ab.transpose(0, 2, 1, 3)
t2bb = t2bb.transpose(0, 2, 1, 3)
print(t1a.shape, t1b.shape)
print(t2aa.shape)
print(t2ab.shape)
print(t2bb.shape)
wave_data['t1a'], wave_data['t1b'] = t1a, t1b
wave_data['t2aa'], wave_data['t2ab'], wave_data['t2bb'] \
    = t2aa, t2ab, t2bb

(1, 1) (1, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)
(1, 1, 1, 1)


In [43]:
etorb, torb = _th12_orb(trial, walker_up, walker_dn, ham_data, wave_data)

In [44]:
print(etorb, torb)

-0.020030467639531698 0.0


In [45]:
lno_result[1]

-0.020030467639531698

In [36]:
walker_up_r = np.random.randn(*walker_up.shape)
walker_dn_r = np.random.randn(*walker_dn.shape)
ecorr = _calc_ecorr(trial, walker_up_r, walker_dn_r, ham_data, wave_data)
eorb = _calc_eorb(trial, walker_up_r, walker_dn_r, ham_data, wave_data)
e_ref = trial._calc_energy(walker_up_r, walker_dn_r, ham_data, wave_data)
print(eorb)
print(ecorr)
print(jnp.real(e_ref) - emf)

0.4091269673929979
0.8149628578785431
0.8149616665566066


In [ ]:
nocca, noccb = trial.nelec
prjloa, prjlob = wave_data['prjlo']
rot_chola, rot_cholb = ham_data['rot_chol']
GFa, GFb = trial._calc_green(walker_up, walker_dn, wave_data)
lga = oe.contract('gij,jk->gik', rot_chola[:,:nocca,nocca:],
                GFa.T[nocca:,:nocca], backend="jax")
lgb = oe.contract('gij,jk->gik', rot_cholb[:,:noccb,noccb:],
                GFb.T[noccb:,:noccb], backend="jax")
tr_lga = oe.contract('gii->g',lga)
tr_lgb = oe.contract('gii->g',lgb)
lga_orb = oe.contract('gik,ik->g',lga,prjloa)
lgb_orb = oe.contract('gik,ik->g',lgb,prjlob)
lglg_aa = oe.contract('g,g->',lga_orb,tr_lga) - oe.contract('gij,gjk,ik->',lga,lga,prjloa)
lglg_ab = oe.contract('g,g->',lga_orb,tr_lgb)
lglg_ba = oe.contract('g,g->',tr_lga,lgb_orb)
lglg_bb = oe.contract('g,g->',lgb_orb,tr_lgb) - oe.contract('gij,gjk,ik->',lgb,lgb,prjlob)
eorb = 0.5*(lglg_aa + lglg_ab + lglg_bb)

In [39]:
print(prjloa)
print(prjlob)

[[ 1.00000000e+00 -5.10962800e-15]
 [-5.10962800e-15  2.61082983e-29]]
[[0. 0.]
 [0. 0.]]


In [54]:
GFa, GFb = trial._calc_green(walker_up_r, walker_dn_r, wave_data)
GFa, GFb = GFa[:nocca, nocca:], GFb[:noccb, noccb:]
ot2aa = oe.contract('iajb,ia,jb->',t2aa,GFa,GFa)
ot2aa_ex = oe.contract('iajb,ib,ja->',t2aa,GFa,GFa)
ot2ab = oe.contract('iajb,ia,jb->',t2ab,GFa,GFb)
ot2ba = oe.contract('jbia,jb,ia->',t2ab,GFa,GFb)
ot2bb = oe.contract('iajb,ia,jb->',t2bb,GFb,GFb)
ot2bb_ex = oe.contract('iajb,ib,ja->',t2bb,GFb,GFb)
print(ot2aa,ot2aa_ex)
print(ot2ab,ot2ba)
print(ot2bb,ot2bb_ex)

-0.09681721373534696 0.09681721373534696
-0.24841543499427354 -0.24841543499427354
-0.19655792219408952 0.19655792219408952


In [58]:
t12 = _t_olp(trial, walker_up, walker_dn, wave_data)
print(t12)

(-0+0j)
